<a href="https://colab.research.google.com/github/mlfigueiredo/CienciaDosDados/blob/main/_NaPr%C3%A1tica_Previs%C3%A3o_de_Sal%C3%A1rio_com_Random_Forest_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##<font color='grey'>Previsão de Salário com Random Forest</font>
 
# <font color='black'>Machine Learning</font>

# 1° Definir o Problema de Negócio

Necessitamos prever quais as pessoas estão no rol daquelas que vão ganhar mais de 50k dolares anuais. As pessoas estão categorizadas entre aquelas que vão ter o salário <=50k ou >50k anuais.

==> **Obs**:  Se por acaso tiver problema com o Pandas Profiling, desinstale e instale novamente com os comandos abaixo:

**Desinstalar**: !pip uninstall pandas-profiling

**Instalar**  :  !pip install pandas-profiling

# 2° Análise Exploratória dos dados


In [6]:
 # Instalação do Pandas Profiling
 !pip install pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl (262 kB)


In [12]:
# Importação das Bibliotecas 
import pandas as pd
#from pandas_profiling import ProfileReport
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [13]:
# Importação dos dados a serem utilizados para a criação do modelo de ML
#Esse banco de dados está na web no github para não ter que ficar importando
data = pd.read_csv('https://raw.githubusercontent.com/llSourcell/Best-Programming-Languages-for-Machine-Learning/master/adults.txt', sep=',')
data


,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
## Pandas Profiling Relatório
from pandas_profiling import ProfileReport

profile = ProfileReport(data)

profile.to_notebook_iframe()

profile.to_file(output_file="Relatório Base de Dados.html")

# 3° Pré-Processamento dos dados


In [14]:
# Convertendo variáveis string em numéricas com o "LabelEncoder"
for variaveis in ['sex', 'race', 'occupation','education', 'workclass', 'relationship']:
    data[variaveis] = LabelEncoder().fit_transform(data[variaveis])

In [15]:
data

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,7,77516,9,13,Never-married,1,1,4,1,2174,0,40,United-States,<=50K
1,50,6,83311,9,13,Married-civ-spouse,4,0,4,1,0,0,13,United-States,<=50K
2,38,4,215646,11,9,Divorced,6,1,4,1,0,0,40,United-States,<=50K
3,53,4,234721,1,7,Married-civ-spouse,6,0,2,1,0,0,40,United-States,<=50K
4,28,4,338409,9,13,Married-civ-spouse,10,5,2,0,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,Married-civ-spouse,13,5,4,0,0,0,38,United-States,<=50K
32557,40,4,154374,11,9,Married-civ-spouse,7,0,4,1,0,0,40,United-States,>50K
32558,58,4,151910,11,9,Widowed,1,4,4,0,0,0,40,United-States,<=50K
32559,22,4,201490,11,9,Never-married,1,3,4,1,0,0,20,United-States,<=50K


In [20]:
# Selecionando as variáveis a serem utilizadas na Previsão e colocando em 'X'
X = data[['sex', 'race', 'hours_per_week', 'occupation', 'education', 'workclass', 'relationship', 'capital_gain']]

# Colocando a variável target (label, variavel a ser predita....) no objeto 'Y'
Y = data['salary'].values.tolist()


In [21]:
# Dividindo os dados em Treino e Teste (30% para teste)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# 4° Criando a Máquina Preditiva


In [22]:
# Criando a Máquina Preditiva - Instanciando o Algoritmo de ML (Chamando o algoritmo a ser usado)
clf_RF = RandomForestClassifier(n_estimators=1000)


In [23]:
# Treinando o Algoritmo com os dados de treino
clf_RF = clf_RF.fit(X_train, Y_train)


# 5° Avaliando a Acurácia do Modelo de Machine Learning

In [24]:
# Validando a Maquina Preditiva - Verificando o nível de Acurácia da Máquina Preditiva
accuracy = clf_RF.score(X_test, Y_test)
print('Accuracy: ' + str(accuracy)) 

# o baseline tem Accuracy: 0.8173815129491248
# incluindo a variável 'capital_gain' melhoramos para  0.8420513870406388

Accuracy: 0.8383662606203296


In [26]:
# Importância das Variáveis
variaveis_importantes = clf_RF.feature_importances_
print(variaveis_importantes)

[0.02504914 0.02546885 0.18035755 0.13133608 0.1456945  0.06811608
 0.23512721 0.18885059]


# 6° Fazendo Novas Previsões com a Máquina Preditiva


In [27]:
# Utilizando a Máquina Preditiva - Fazendo Previsões com novos dados 
Novas_Previsoes = clf_RF.predict(X_test)
print(Novas_Previsoes)

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '<=50K']


In [28]:
Novas_Previsoes

array(['<=50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '<=50K'],
      dtype='<U5')